# Importanción de librerias y configuraciones iniciales

In [ ]:
import numpy as np # Importo numpy
import pandas as pd # Importo pandas
pd.options.display.max_rows = 20 # maximo 20 filas en la salida
pd.options.display.max_colwidth = 80 # anchura maxima de 80 caracteres de texto a mostrarse
pd.options.display.max_columns = 20 # maximo 20 columnas en la salida
import matplotlib.pyplot as plt
plt.rc("figure", figsize=(10, 6)) # define la configuracion de los graficos
np.set_printoptions(precision=4, suppress=True) # configuracion de numpy 4 decimales y evita notacion cientifica

# Carga de datos
Usaremos dos datasets, los cuales contienen los resultados de las elecciones presidenciales de 2019 y 2023 en la provincia del Chaco.

In [ ]:
# Cargo los datos de los datasets
df19 = pd.read_csv("resultados_presidenciales_2019.csv") # Resultados electorales 2019
df23 = pd.read_csv("resultados_presidenciales_2023.csv") # Resultados electorales 2023

# Exploración Inicial

Usaremos los datos electorales del 2023 (**df23**) para esta demostracion.

In [ ]:
# Veamos las primeras filas
df23.head()

In [ ]:
# Veamos la informacion de las columnas
df19.info()

# 1. Indexación Jerárquica
Indexamos por sección electoral y por agrupación política, y luego renombramos y ordenamos los índices.

In [ ]:
# Indexamos los datos por seccion y agrupacion
df19 = df19.set_index(["seccion_nombre", "agrupacion_nombre"]) # Indexacion de columnas
df23 = df23.set_index(["seccion_nombre", "agrupacion_nombre"]) # Indexacion de columnas
# Renombramos los indices
df19.index.names = ["Sección", "Agrupación"] # Renombramos los indices
df23.index.names = ["Sección", "Agrupación"] # Renombramos los indices
# Ordenamos los datasets
df19 = df19.sort_index(level=["Agrupación"]) # Ordenamos por Agrupación 
df23 = df23.sort_index(level=["Agrupación"]) # Ordenamos por Agrupación 
# Mostramos el datasets de resultados electorales del 2023
df23.head(10) # Mostramos las primeras 10 filas del distrito Chaco


# 2. Estadisticas
Sumamos la cantidad de votos por agrupación política y reflejamos los totales en un gráfico de barras.

In [ ]:
# Renombramos undefined por Otros en el nivel de Agrupación
df23 = df23.rename(index={"undefined": "Otros"}, level="Agrupación")
# Agrupamos por agrupación y sumamos los votos, ordenando de mayor a menor
df23.groupby(level="Agrupación")["votos_cantidad"].sum().sort_values(ascending=False)
# Graficamos los resultados en un gráfico de barras y mostramos el grafico y tabla
df23.groupby(level="Agrupación")["votos_cantidad"].sum().sort_values(ascending=False).plot.bar(title="Resultados Electorales Presidenciales 2023 - Chaco", ylabel="Cantidad de votos", xlabel="Agrupación")
# Agrego etiquetas a las barras
for i, v in enumerate(df23.groupby(level="Agrupación")["votos_cantidad"].sum().sort_values(ascending=False)):
    plt.text(i, v + 1000, str(v), ha='center', va='bottom', fontsize=8)
# Muestro el grafico
plt.show()

# 3. Fusion segun el indice

Se cuentan los votos positivos por cada sección en cada año y luego se arma una tabla comparativa entre los años 2019 y 2023 para ver la diferencia de votos por cada sección.

In [ ]:
# Recargo los datos de los datasets
df19 = pd.read_csv("resultados_presidenciales_2019.csv") # Resultados electorales 2019
df23 = pd.read_csv("resultados_presidenciales_2023.csv") # Resultados electorales 2023

# Cuentamos la cantidad de votos positivos de cada dataset
df19 = df19[df19["votos_tipo"].str.contains("Positivo", case=False)]
df23 = df23[df23["votos_tipo"].str.contains("Positivo", case=False)]

# Agrupamos por sección(departamento electoral) y sumamos los votos
v2019 = (df19.groupby("seccion_id", as_index=False)
             .agg({"votos_cantidad":"sum", "seccion_nombre":"first"})
             .rename(columns={"votos_cantidad":"votos_2019"}))

v2023 = (df23.groupby("seccion_id", as_index=False)
             .agg({"votos_cantidad":"sum", "seccion_nombre":"first"})
             .rename(columns={"votos_cantidad":"votos_2023"}))

# Juntamos ambos datasets por sección y calculamos la diferencia de votos
cmp = pd.merge(v2019,v2023, on=["seccion_id"], how="outer", suffixes=("_2019","_2023"))
cmp["dif_2023_vs_2019"] = cmp["votos_2023"] - cmp["votos_2019"]

# Ordenamos por seccion_id 
cmp = cmp.sort_values("seccion_id")

# Grafico de barras comparando los votos de 2019 y 2023 por sección
cmp.set_index("seccion_nombre_2019")[["votos_2019","votos_2023"]].plot.bar(title="Comparación de votos por sección en Chaco", xlabel="Sección", ylabel="Cantidad de votos")
plt.show()
cmp[["seccion_id","seccion_nombre_2019","votos_2019","votos_2023","dif_2023_vs_2019"]]


# 4. Concatenación

Concatenar los datasets de 2019 y 2023, y luego analizar cómo cambiaron los votos por agrupación política en la sección de San Fernando.

In [ ]:
df_total = pd.concat([df19, df23], ignore_index=True) # Concatenamos ambos datasets
df_chaco = df_total[df_total['seccion_nombre'] == 'San Fernando'] # Filtramos por el departamento San Fernando del Chaco
resumen = df_chaco.groupby(['agrupacion_nombre', 'año'])['votos_cantidad'].sum().unstack() # Agrupamos por agrupacion y año, sumando los votos y deshaciendo el multiindice
# hacer un gráfico de barras horizontal para comparar los votos de cada agrupación en 2019 y 2023
resumen.plot.barh(title="Comparación de votos por agrupación en San Fernando", xlabel="Cantidad de votos", ylabel="Agrupación")
# Agrego etiquetas a las barras y truncando los valores a enteros
for i, (v2019, v2023) in enumerate(zip(resumen[2019], resumen[2023])):
    plt.text(v2019 + 100, i - 0.2, str(v2019), ha='center', va='bottom', fontsize=8)
    plt.text(v2023 + 100, i + 0.2, str(v2023), ha='center', va='bottom', fontsize=8)
# Muestro el grafico
plt.show()
# Muestro la tabla resumen
resumen

# 5 Remodelación
Mostramos la cantidad de votos por sección y agrupación, para cada tipo de voto

In [ ]:
# Recargo los datos del dataset de 2023 que vamos a usar como ejemplo
df23 = pd.read_csv("resultados_presidenciales_2023.csv") # Resultados electorales 2023
# Convertir el DataFrame df23 de formato largo a formato ancho (pivot)
# Nos ayudamos de la indexacción jerarquica previa
pivot_df = df23.pivot_table(
    index=["seccion_nombre", "agrupacion_nombre"],
    columns="votos_tipo",
    values="votos_cantidad",
    aggfunc="sum",
    fill_value=0
)

# Mostramos las primeras filas del DataFrame remodelado
pivot_df

In [ ]:
# Convertimos el DataFrame pivot_df de formato ancho a formato largo (unstack -> stack -> reset_index)
pivot_long = pivot_df.stack().reset_index() # Convertimos el DataFrame a formato largo
pivot_long.columns = ['Sección', 'Agrupación', 'votos_tipo', 'votos_cantidad'] # Renombramos las columnas
# Renombramos undefined por Otros en el nivel de Agrupación para que coincida con el dataset de 2023
pivot_long = pivot_long.replace('undefined', 'Otros')
pivot_long

# 5 Transposición

In [ ]:
# Ejemplo de transposición: intercambiar filas y columnas en el DataFrame pivot_df
pivot_df_transposed = pivot_df.transpose()

# Mostramos las primeras filas del DataFrame transpuesto
pivot_df_transposed

# Desafio 

Tomando el dataset de **resultados 2023** en Chaco, realizá un análisis de los votos positivos por agrupación política en cada sección electoral.

🐼 Filtrá solo los votos de **tipo POSITIVO**.

🐼 Creá un **índice jerárquico (MultiIndex)** con **seccion_nombre** y **agrupacion_nombre**.

🐼 Usá **groupby** para obtener la cantidad total de votos positivos por agrupación en cada sección.

🐼 Calculá el **total de electores** por sección.

🐼 Hacé un **merge** para unir ambos resultados en un mismo DataFrame.

🐼 Agregá una columna con el porcentaje de votos positivos sobre el total de electores de cada sección.